In [ ]:
import os  
import tarfile  
import requests  
import torchvision.transforms as transforms  
import torchvision.datasets as datasets  
from torch.utils.data import DataLoader  
import torch.optim as optim  

# Step 1: Download and extract the dataset  
dataset_url = "http://diode-dataset.s3.amazonaws.com/val.tar.gz"  
dataset_dir = './diode_dataset'  

if not os.path.exists(dataset_dir):  
    os.makedirs(dataset_dir)  

# Download the dataset  
response = requests.get(dataset_url)  
with open(os.path.join(dataset_dir, 'val.tar.gz'), 'wb') as f:  
    f.write(response.content)  

# Extract the dataset  
with tarfile.open(os.path.join(dataset_dir, 'val.tar.gz'), 'r:gz') as tar:  
    tar.extractall(path=dataset_dir)  

# Step 2: Set up the model and dataset  
class ConvBlock(nn.Module):  
    # ... (rest of ConvBlock class remains unchanged)  

class RegNet(nn.Module):  
    # ... (rest of RegNet class remains unchanged)  

# Initialize model  
num_classes = len(os.listdir(dataset_dir))  # Assuming each subdir is a class  
model = RegNet(num_classes=num_classes)  

# Step 3: Data loading and preprocessing  
transform = transforms.Compose([  
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
])  

# Assuming the dataset has subdirectories for each class  
dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  

# Step 4: Training setup  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model.to(device)  

criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# Step 5: Training loop  
num_epochs = 10  # Adjust as necessary  
for epoch in range(num_epochs):  
    model.train()  
    running_loss = 0.0  

    for images, labels in dataloader:  
        images, labels = images.to(device), labels.to(device)  

        optimizer.zero_grad()  # Zero gradients  
        outputs = model(images)  # Forward pass  
        loss = criterion(outputs, labels)  # Compute loss  
        loss.backward()  # Backpropagation  
        optimizer.step()  # Optimize  

        running_loss += loss.item()  

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")  

# Step 6: Save the model  
torch.save(model.state_dict(), 'regnet.pth')  
print("Model saved.")  

# Optional: Validate the model or visualize predictions